# IFT 6135 A2

In [196]:
import csv
import torch.cuda
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
from torchvision import transforms, datasets
from torch.distributions import * 
from os.path import dirname, abspath, join
from torch import norm
import config
import numpy as np
import pickle as pk


In [197]:
cuda_available = torch.cuda.is_available()
print(cuda_available)

False


In [198]:
parent_dir = dirname(dirname(abspath('__file__')))
yaml_file = join(parent_dir, 'config.yaml')
config = config.Configuration('Q1_3', yaml_file)

print(config)

config type: Q1_3 
model type: MLPb 
weight initialisation: normal 
learning rate: 0.02 
batch size: 64 
number of epochs: 100
weight decay: 0 
batch norm: False 
fig caption: dropout, p=0.5 



## Load Data

In [199]:
def load_dataset(config):
    mnist_train = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
    mnist_test = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)
    
    test_sampler, valid_sampler = test_valid_split(mnist_test, config)
    
    trainloader = DataLoader(mnist_train, batch_size=config.batch_size, shuffle=True, num_workers=2)
    testloader = DataLoader(mnist_test, batch_size=64, sampler=test_sampler, num_workers=2)
    validloader = DataLoader(mnist_test, batch_size=64, sampler=valid_sampler, num_workers=2)
    return trainloader, testloader, validloader

def test_valid_split(test, config):
    num_test = len(test[0])
    indices = list(range(num_test))
    split = int(np.floor(num_test / 2))

    # split test set into validation and test set
    valid_idx, test_idx = indices[split:], indices[:split]

    valid_sampler = SubsetRandomSampler(valid_idx)
    test_sampler = SubsetRandomSampler(test_idx)

    return test_sampler, valid_sampler

## Define model

In [200]:
class MLPa(nn.Module):
    def __init__(self):
        super(MLPa, self).__init__()
        self.config = config
        self.model = nn.Sequential(
            nn.Linear(784, 600),
            nn.ReLU(),
            nn.Linear(600, 200),
            nn.ReLU(),
            nn.Linear(200, 10), 
            nn.Softmax(dim=1))
        
    def forward(self, x):
        output = self.model(x)
        return output

In [201]:
class MLPb(nn.Module):
    def __init__(self):
        super(MLPb, self).__init__()
        self.config = config
        self.fc1 = nn.Linear(784, 600)
        self.fc2 = nn.Linear(600, 200)
        self.dropout = nn.Dropout(p=0.5) #last layer dropout
        self.fc3 = nn.Linear(200, 10)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.dropout(x))
        x = F.softmax(self.fc3(x))
        return x
    

In [202]:
class CNNa(nn.Module):
    def __init__(self):
        super(CNNa, self).__init__()
        self.conv = nn.Sequential(
            # Layer 1
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3, 3), padding=1),
            nn.Dropout(p=0.5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 2
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), padding=1),
            nn.Dropout(p=0.5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 3
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1),
            nn.Dropout(p=0.5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 4
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1),
            nn.Dropout(p=0.5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2)
        )
        # Logistic Regression
        self.clf = nn.Linear(128, 10)

    def forward(self, x):
        return self.clf(self.conv(x).squeeze())   

In [203]:
class CNNb(nn.Module):
    def __init__(self):
        super(CNNb, self).__init__()
        self.conv = nn.Sequential(
            # Layer 1
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(p=0.5),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 2
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(p=0.5),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 3
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Dropout(p=0.5),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # Layer 4
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Dropout(p=0.5),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2)
        )
        # Logistic Regression
        self.clf = nn.Linear(128, 10)

    def forward(self, x):
        return self.clf(self.conv(x).squeeze())   

In [204]:
def build_model():
    if config.model_type == 'MLPa':
        model = MLPa()
    elif config.model_type == 'MLPb':
        model = MLPb()
    elif config.model_type == 'CNN':
        if not config.batch_norm:
            model = CNNa()
        else:
            model = CNNb()

    if torch.cuda.is_available():
        model = model.cuda()
         
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=config.lr0, weight_decay = (config.weight_decay/9912422)*config.batch_size)
    return model, criterion, optimizer

## Train model

In [205]:

def train_model(config, model, criterion, optimizer):
    losses = []
    parameter_norms = []
    train_losses = []
    test_losses = []
    # record the performance for this epoch
    trainloader, testloader, validloader = load_dataset(config)
    
    for epoch in range(config.num_epochs):
        for batch_idx, (inputs, targets) in enumerate(trainloader):
            parameter_norm = []
            optimizer.zero_grad()
             
            if config.model_type == 'MLPa' or config.model_type == 'MLPb':
                inputs = Variable(inputs).view(-1,784)
                targets = Variable(targets).view(-1)
            elif config.model_type == 'CNN':
                inputs = Variable(inputs).view(-1,1,28,28)
                targets = Variable(targets).view(-1)
            if cuda_available:
                inputs, targets = inputs.cuda(), targets.cuda()
            
            optimizer.zero_grad()
            # compute loss
            loss = criterion(model(inputs), targets)
            loss.backward()
            optimizer.step()
            losses.append(loss.data[0])
            if config.config_type == 'Q1_1' or config.config_type == 'Q1_2':
                for param in model.parameters():
                    parameter_norm.append(norm(param))
            parameter_norms.append(parameter_norm)
        # print the results for this epoch
        test_loss = find_testloss(model, testloader)
        test_losses.append(test_loss)
        loss_at_epoch = np.mean(losses)
        train_losses.append(loss_at_epoch)
        print("Epoch {0} \n Train Loss : {1:.3f} \Test Loss : {2:.3f}".format(epoch, loss_at_epoch, test_loss))
    
    test_loss_filename = 'test_loss_' + config.config_type + '.pk'
    train_loss_filename = 'train_loss_' + config.config_type + '.pk'
    pk.dump(test_losses, open(join(parent_dir, test_loss_filename), 'wb'))
    pk.dump(train_losses, open(join(parent_dir, train_loss_filename),'wb'))

    if config.config_type == 'Q1_1' or config.config_type == 'Q1_2':
        params_filename = 'parameters_' + config.config_type + '.pk'
        pk.dump(parameter_norms, open(join(parent_dir, params_filename), 'wb'))
    
    print("Finished training Model {}".format(config.config_type))
    

In [206]:
def find_testloss(model,test_loader):
    model.eval() 
    test_loss_iter = []
    for data in test_loader:
        inputs, targets = data

        if config.model_type == 'MLPa' or config.model_type == 'MLPb':
            inputs = Variable(inputs, volatile=True).view(-1,784)
            targets = Variable(targets, volatile=True).view(-1)
        elif config.model_type == 'CNN':
            inputs = Variable(inputs, volatile=True).view(-1,1,28,28)
            targets = Variable(targets, volatile=True).view(-1)
        if cuda_available:
            inputs, targets = inputs.cuda(), targets.cuda()

        outputs = model(inputs)
        test_loss = criterion(outputs, targets)
        test_loss_iter.append(test_loss)
    iteration_test_loss = np.mean(test_loss_iter)
    return iteration_test_loss.data[0]

In [207]:
def dropout_validate(model, N, validloader):
    model.eval()
    mask_probs = np.ones(600)*0.5
    bern = Bernoulli(torch.from_numpy(mask_probs))
    
    correct = 0
    total = 0
    
    for x, y in validloader:
        
        x = Variable(x, volatile=True).view(-1,784)
        y = Variable(y, volatile=True).view(-1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        for i in range(N):
            dropout_mask = bern.sample()
            sum_out += F.relu(x*dropout_mask)
        x = sum_out / N        
        outputs = F.Softmax(self.fc3(x)) 
        
        _, predicted = torch.max(outputs.data, 1)
        total += y.size(0)
        correct += (predicted == y).sum()
    
    return correct/total

In [208]:
model, criterion, optimizer = build_model()
train_model(config, model, criterion, optimizer)

if config.config_type == 'Q1_4' or config.config_type == 'Q1_5':
    dropout_validates = []
    ns = [i*10 for i in range(1,11)]
    for n in ns:
        dv = dropout_validate(model, n, validloader)
        dropout_validates.append(dv)
    dropout_validate_filename = 'dropoutvalidation_' + config.config_type + '.pk'
    pk.dump(dropout_validates, open(join(parent_dir, dropout_validate_filename), 'wb'))

/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


2.3031275272369385
2.3030712604522705
2.30216121673584
2.3018925189971924
2.3021156787872314
2.301877021789551
2.3021910190582275
2.301276683807373
2.302866220474243
2.3022425174713135
2.302049398422241
2.3022916316986084
2.3026187419891357
2.302224636077881
2.302543878555298
2.3019769191741943
2.3019790649414062
2.3024682998657227
2.303680181503296
2.302198886871338
2.302255153656006
2.3020613193511963
2.302912712097168
2.3023173809051514
2.302852153778076
2.3028876781463623
2.3022775650024414
2.301521062850952
2.30212140083313
2.302036762237549
2.3021974563598633
2.301361322402954
2.3014912605285645
2.3023922443389893
2.3018226623535156
2.3024985790252686
2.3017935752868652
2.3019063472747803
2.301762342453003
2.3017876148223877
2.3023197650909424
2.3028154373168945
2.3019306659698486
2.30222487449646
2.3021082878112793
2.3025319576263428
2.3014628887176514
2.301170825958252
2.3028552532196045
2.3009774684906006
2.3016130924224854
2.302762031555176
2.301567554473877
2.301396608352661

Process Process-267:
Traceback (most recent call last):
  File "/anaconda/envs/py36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/anaconda/envs/py36/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda/envs/py36/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 55, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/anaconda/envs/py36/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 55, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/anaconda/envs/py36/lib/python3.6/site-packages/torchvision-0.2.0-py3.6.egg/torchvision/datasets/mnist.py", line 73, in __getitem__
    img = Image.fromarray(img.numpy(), mode='L')
  File "/anaconda/envs/py36/lib/python3.6/site-packages/PIL/Image.py", line 2445, in fromarray
    return frombuffer(mode, size, obj, "raw", rawmode, 0, 1)
Proces

1.6444329023361206
1.7414766550064087


  File "/anaconda/envs/py36/lib/python3.6/site-packages/PIL/Image.py", line 2391, in frombuffer
    im = new(mode, (1, 1))
  File "/anaconda/envs/py36/lib/python3.6/site-packages/PIL/Image.py", line 2294, in new
    return Image()._new(core.fill(mode, size, color))
Traceback (most recent call last):
  File "/anaconda/envs/py36/lib/python3.6/site-packages/PIL/Image.py", line 550, in _new
    new.info = self.info.copy()
  File "/anaconda/envs/py36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
KeyboardInterrupt
  File "/anaconda/envs/py36/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda/envs/py36/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 50, in _worker_loop
    r = index_queue.get()
  File "/anaconda/envs/py36/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/anaconda/envs/py36/lib/python3.6/multiprocessing/synchroniz

KeyboardInterrupt: 

In [ ]:
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('-f', '--configfile', help='config file in yaml format', required=True)
    parser.add_argument('-r', '--configtype', help='configurations to run', required=True)
    args = parser.parse_args()
    return vars(args)